###This script is a port of EnergyID's code that calculates a linear regression on heating data

# Imports and setup

In [ ]:
import sys, os, inspect
import pandas as pd

In [ ]:
script_dir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
# add the path to opengrid to sys.path
sys.path.append(os.path.join(script_dir, os.pardir, os.pardir))

from opengrid.library import config
c=config.Config()
DEV = c.get('env', 'type') == 'dev' # DEV is True if we are in development environment, False if on the droplet

import matplotlib.pyplot as plt

# find tmpo
sys.path.append(c.get('tmpo', 'folder'))

from opengrid.library.houseprint import houseprint

if DEV:
    if c.get('env', 'plots') == 'inline':
        %matplotlib inline
    else:
        %matplotlib qt
else:
    pass # don't try to render plots
plt.rcParams['figure.figsize'] = 12,8

# Load Test Data

##Load Degree Days

In [ ]:
dfWeather = pd.read_csv('WeatherData.csv')

In [ ]:
dfWeather = dfWeather.set_index(pd.DatetimeIndex(dfWeather['Date']))

In [ ]:
tsDegreeDays = dfWeather['DegreeDays']

In [ ]:
tsDegreeDays.name = 'x'

In [ ]:
tsDegreeDays = tsDegreeDays.resample(rule='MS',how='sum')

##Load Measurements

In [ ]:
import json
import numpy as np

In [ ]:
json_data = open('diederik.json')

In [ ]:
js = json.load(json_data)

In [ ]:
#make two arrays with data
X = [point['x']/1000 for point in js['points']]
Y = [point['y'] for point in js['points']]

In [ ]:
#convert unix timestamp to datetime
X = np.array(X).astype('datetime64[s]')

In [ ]:
tsEnergyData = pd.Series(data=Y,index=pd.DatetimeIndex(X), name='y')

###Bring weather data and energy data together

In [ ]:
data = pd.concat([tsDegreeDays,tsEnergyData],axis=1).dropna()

In [ ]:
print data

# Analysis Definition

In [ ]:
from opengrid.library.analyses.analysis import Analysis

In [ ]:
from scipy import stats

class LinearRegression(Analysis):
    """
        Calculate a simple linear regression given a dataframe with X and Y values
    """
    def __init__(self,data):
        """
            Parameters
            ----------
            data: Pandas Dataframe
                This dataframe has to be strictly formatted!
                One column must be named 'x', the other one 'y'
        """

        super(LinearRegression, self).__init__()

        self.data = data

        self.slope, self.intercept, self.r_value, self.p_value, self.std_err = stats.linregress(data['x'],data['y'])

    def getY(self,x):
        """
            Calculate the value on the trend line for a given x-value or an array of x-values

            Parameters
            ----------
            x: single number (float or int)
                OR iterable (array) of numbers

            Returns
            -------
            float if x is a single number
            array of floats if x is an iterable
        """

        #check if x is an iterable
        if not hasattr(x, '__iter__'):
            return self._getY(x)
        else:
            return [self._getY(val) for val in x]

    def _getY(self,x):
        """
            Calculate a value on the trend line for a given x-value

            Parameters
            ----------
            x: number

            Returns
            -------
            float
        """

        # y = ax + b
        return self.slope * x + self.intercept

    def getX(self,y):
        """
            Calculate value on the trendline for given y-value

            Parameters
            ----------
            y: number

            Return
            ------
            float
        """

        # y = ax + b => x = (y-b)/a
        return (y - self.intercept)/self.slope

    def toPlt(self, forceOrigin=True):
        """
            Create scatterplot and plot trendline

            Parameters
            ----------
            forceOrigin: boolean, default True
                set the default values of the axis to 0,0

            Returns
            -------
            matplotlib figure
        """
        fig = plt.figure()
        ax1 = fig.add_subplot(111)

        ax1.scatter(self.data['x'], self.data['y'], alpha=1, s=20)

        function = "${s:.2f}x + {i:.2f}$".format(s = self.slope, i = self.intercept)
        r2 = "$R^2$: ${r:.2f}$".format(r = self.r_value)
        label = "{}\n{}".format(function,r2)

        x = self._getPlotX()

        ax1.plot(x, self.getY(x), '-', label=label)
        plt.legend()

        if forceOrigin:
            x1,x2,y1,y2 = plt.axis()
            plt.axis((0,x2,0,y2))

        return fig

    def _getPlotX(self):
        """
            Return the values that are needed to draw the trendline.
            In this case, the min and max since it's a single line
        """

        return [self.data['x'].min(),self.data['x'].max()]

In [ ]:
class LinearRegression2(LinearRegression):
    """
        Calculate a linear regression that uses a predefined breakpoint to break between regression and baseload
    """
    def __init__(self,data,breakpoint):
        """
            Parameters
            ----------
            data: Pandas Dataframe
                columns must be exactly named 'x' and 'y'
            breakpoint: number
                value on the x-axis where to break between regression and baseload
        """
        
        self.breakpoint = breakpoint
        
        #baseLoad is the mean of all y values where x is below the breakpoint
        self.baseLoad = data[data['x']<=self.breakpoint]['y'].mean()
        
        #calculate trendline by making a simple linear regression (superclass) on the right hand side data
        super(LinearRegression2, self).__init__(data = self._calculateRegressionData(data))
        
        #intersect is the intersection of the trendline and the baseload
        self.intersect = self.getX(self.baseLoad)
        
        #the super init writes only right hand side data to self.data, so we have to overwrite it after initialisation
        self.data = data
        
    def _calculateRegressionData(self, data):
        """
            Decide what data to use for the linear regression.
            In this case all data past the breakpoint
        """
        
        return data[data['x']>self.breakpoint]
        
    def _getY(self,x):
        """
            Calculate a value on the trend line for a given x-value
            
            Parameters
            ----------
            x: number
            
            Returns
            -------
            float
        """
        #if the value is before the intersection, return the baseload
        if x <= self.intersect:
            return self.baseLoad
        else:
            #else return the normal value in the trendline
            return super(LinearRegression2, self)._getY(x)
        
    def _getPlotX(self):
        """
            Return the values that are needed to draw the trendline.
            In this case, they are the same as the normal linear regression,
            but the intersection of the trend and the baseload need to be added
        """
        ret = super(LinearRegression2, self)._getPlotX()
        ret.append(self.intersect)
        return sorted(ret)

In [ ]:
class LinearRegression3(LinearRegression2):
    """
        Calculate a linear regression that uses a predefined breakpoint to break between baseload and regression,
        yet exclude values in a certain range (percentage of the baseload) from the regression
    """
    
    def __init__(self, data, breakpoint, percentage, includeEndOfBaseLoadInRegression=True):
        """
            Parameters
            ----------
            
            data: Pandas Dataframe
                columns must be named 'x' and 'y'
            breakpoint: number
                point on the x-axis where to break between baseload and regression
            percentage: float
                y-values that are in this range to the baseload are excluded from the regression
            includeEndOfBaseLoadInRegression: boolean
                
        """
        
        self.percentage = percentage
        self.includeEndOfBaseLoadInRegression = includeEndOfBaseLoadInRegression
        
        super(LinearRegression3, self).__init__(data=data, breakpoint=breakpoint)
        
    def _calculateRegressionData(self,data):
        """
            Decide what data to use for the linear regression.
            In this case all data past the breakpoint (from Linearregression2),
            but we iterate over them and drop values that are close to the baseline.
        """
        #make a list of indices of entries that are to be excluded from the regression
        toDrop = data[data['x']<=self.breakpoint].sort('x').index.tolist()
        
        #get the data from Regression 3
        res = super(LinearRegression3, self)._calculateRegressionData(data)
        
        #sort by x-value and iterate
        for entry in res.sort('x').iterrows():
            #if the y value is smaller than the percentage of the baseload
            if entry[1]['y'] < self.baseLoad * (1+self.percentage):
                #add the entry to the list to be dropped
                toDrop.append(entry[0])
            else:
                #if not, end the loop
                break
                
        # if we want to include the last value of the base load in the regression, remove it from the todrop list
        if self.includeEndOfBaseLoadInRegression:
            toDrop.pop()
                
        #drop the toDrop list from the dataframe
        res = data.drop(toDrop)
        
        return res

#Run Analysis

In [ ]:
linregress = LinearRegression(data)

In [ ]:
fig = linregress.toPlt()

In [ ]:
linregress2 = LinearRegression2(data=data, breakpoint=60)

In [ ]:
fig = linregress2.toPlt()

In [ ]:
linregress3 = LinearRegression3(data=data, breakpoint=60, percentage=2, includeEndOfBaseLoadInRegression=True)

In [ ]:
fig = linregress3.toPlt()

#Plot Models

In [ ]:
fig = plt.figure()
ax1 = fig.add_subplot(111)
ax1.plot_date(data.index, data['x'],'-',label='gas')
ax1.plot_date(data.index, linregress.getY(data['x']),'-', label='simple regression')
ax1.plot_date(data.index, linregress2.getY(data['x']),'-', label='with baseline')
ax1.plot_date(data.index, linregress3.getY(data['x']),'-', label='progressive baseline')
plt.legend()